In [1]:
# Title:

In [26]:
# packages
import datetime as dt
import pandas as pd

import model
import utils

May need to change to fit other strategy


In [20]:
from loguru import logger
import typing as t

from quantified_strategies.event_trading import russell_rebalance, super_bowl
from quantified_strategies.overnight_trading import short_term_reversal
from quantified_strategies.seasonal_trading import (
    buy_when_yields_are_low, pay_day_strategy, santa_claus_strategy, september_bear, tax_day_strategy, turn_around_tuesday_strategy)
from quantified_strategies.technical_trading import contrarian_trading

from quantified_strategies import strategy_utils

def get_active_strategies(strategy_map: t.Dict[str, str | t.List[str]], date: dt.date, verbose: bool = True) -> t.Dict[str, t.List[str]]:
    
    strategy_map_list = {strat: universe if isinstance(universe, list) else [universe] for strat, universe in strategy_map.items()}

    active_strats = {strat: [] for strat in strategy_map.keys()}
    
    if (tickers := strategy_map_list.get("russell_rebalance")) and russell_rebalance.activity.is_active(date=date):
        if verbose:
            logger.info(f"Active: 'russell_rebalance' -> {','.join(tickers)}")
        active_strats["russell_rebalance"] += tickers
        
    if (tickers := strategy_map_list.get("super_bowl")) and super_bowl.activity.is_active(date=date):
        if verbose:
            logger.info(f"Active: 'super_bowl' -> {','.join(tickers)}")
        active_strats["super_bowl"] += tickers
        
    if tickers := strategy_map_list.get("short_term_reversal"):
        for ticker in tickers:
            if short_term_reversal.activity.is_active(data=short_term_reversal.utils.fetch(ticker=ticker), date=date):
                if verbose:
                    logger.info(f"Active: 'short_term_reversal' -> {ticker!r}")
                active_strats["short_term_reversal"].append(ticker)
        
    if (tickers := strategy_map_list.get("buy_when_yields_are_low")) and buy_when_yields_are_low.activity.is_active(treasury=buy_when_yields_are_low.utils.fetch_treasury(), date=date, window=15):
        if verbose:
            logger.info(f"Active: 'buy_when_yields_are_low' -> {','.join(tickers)}")
        active_strats["buy_when_yields_are_low"] += tickers
        
    if (tickers := strategy_map_list.get("pay_day_strategy")) and pay_day_strategy.activity.is_active(date=date):
        if verbose:
            logger.info(f"Active: 'pay_day_strategy' -> {','.join(tickers)}")
        active_strats["pay_day_strategy"] += tickers
        
    if (tickers := strategy_map_list.get("sankt_nikolaus_strategy")) and santa_claus_strategy.sankt_nikolaus.activity.is_active(date=date):
        if verbose:
            logger.info(f"Active: 'sankt_nikolaus_strategy' -> {','.join(tickers)}")
        active_strats["sankt_nikolaus_strategy"] += tickers
        
    if (tickers := strategy_map_list.get("santa_claus_strategy")) and santa_claus_strategy.santa_claus.activity.is_active(date=date):
        if verbose:
            logger.info(f"Active: 'santa_claus_strategy' -> {','.join(tickers)}")
        active_strats["santa_claus_strategy"] += tickers
        
    if (tickers := strategy_map_list.get("september_bear")) and september_bear.activity.is_active(data=None, date=date, xday=4, week_enter=3, cover_duration=1):
        if verbose:
            logger.info(f"Active: 'september_bear' -> {','.join(tickers)}")
        active_strats["september_bear"] += tickers
        
    if (tickers := strategy_map_list.get("tax_day_strategy")) and tax_day_strategy.activity.is_active(data=None, date=date, enter_before=dict(month=3, day=31), exit_after=dict(month=4, day=15)):
        if verbose:
            logger.info(f"Active: 'tax_day_strategy' -> {','.join(tickers)}")
        active_strats["tax_day_strategy"] += tickers
        
    if tickers := strategy_map_list.get("turn_around_tuesday_strategy"):
        for ticker in tickers:
            if turn_around_tuesday_strategy.activity.is_active(data=turn_around_tuesday_strategy.utils.fetch(ticker=ticker), date=date):
                if verbose:
                    logger.info(f"Active: 'turn_around_tuesday_strategy' -> {ticker!r}")
                active_strats["turn_around_tuesday_strategy"].append(ticker)

    active_strats = {strat: tickers for strat, tickers in active_strats.items() if len(tickers)}

    return active_strats


In [59]:
TRADABLES = ["VUSA.L", "CNX1.L"]
ASSETS = TRADABLES + [utils.CASH]
calc_date = dt.date(2024, 3, 1)

STRATEGY_UNIVERSE_MAP = {
    "super_bowl": TRADABLES,
    "short_term_reversal": TRADABLES,
    # "buy_when_yields_are_low": TRADABLES,
    "pay_day_strategy": TRADABLES,
    "santa_claus_strategy": TRADABLES,
    "september_bear": TRADABLES,
    "tax_day_strategy": TRADABLES,
    "turn_around_tuesday_strategy": TRADABLES,
}
STRATEGY_UNIVERSE_MAP

{'super_bowl': ['VUSA.L', 'CNX1.L'],
 'short_term_reversal': ['VUSA.L', 'CNX1.L'],
 'pay_day_strategy': ['VUSA.L', 'CNX1.L'],
 'santa_claus_strategy': ['VUSA.L', 'CNX1.L'],
 'september_bear': ['VUSA.L', 'CNX1.L'],
 'tax_day_strategy': ['VUSA.L', 'CNX1.L'],
 'turn_around_tuesday_strategy': ['VUSA.L', 'CNX1.L']}

In [60]:
# Fetch active strategies
active_strategies = get_active_strategies(strategy_map=STRATEGY_UNIVERSE_MAP, date=calc_date)
active_strategies

2024-03-02 12:45:05.460 | INFO     | __main__:get_active_strategies:42 - Active: 'pay_day_strategy' -> VUSA.L,CNX1.L


{'pay_day_strategy': ['VUSA.L', 'CNX1.L']}

In [61]:
active_strategies

{'pay_day_strategy': ['VUSA.L', 'CNX1.L']}

In [62]:
X_raw = pd.concat([pd.DataFrame(0, index=[calc_date], columns=pd.MultiIndex.from_product([[key], values])) for key, values in STRATEGY_UNIVERSE_MAP.items()], axis=1)
for strat_name, strat_assets in active_strategies.items():
    for strat_asset in strat_assets:
        X_raw.loc[calc_date, (X_raw.columns.get_level_values(0) == strat_name) & (X_raw.columns.get_level_values(1) == strat_asset)] = 1
X_raw

super_bowl        short_term_reversal        pay_day_strategy  \
               VUSA.L CNX1.L              VUSA.L CNX1.L           VUSA.L   
2024-03-01          0      0                   0      0                1   

                  santa_claus_strategy        september_bear         \
           CNX1.L               VUSA.L CNX1.L         VUSA.L CNX1.L   
2024-03-01      1                    0      0              0      0   

           tax_day_strategy        turn_around_tuesday_strategy         
                     VUSA.L CNX1.L                       VUSA.L CNX1.L  
2024-03-01                0      0                            0      0

In [63]:
my_net = model.Net.load(input_shape=X_raw.shape[1]+len(ASSETS), output_shape=len(ASSETS), name="latest")
my_net

Net(
  (fc0): Linear(in_features=17, out_features=8, bias=True)
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc_output): Linear(in_features=8, out_features=3, bias=True)
)

In [64]:
from torch import nn
from quantified_strategies import ml_utils

def get_allocation(net: nn.Module, X: pd.DataFrame, last_pos: pd.DataFrame):
    
    x = pd.concat([X, last_pos], axis=1)
    
    x_tensor, _ = ml_utils.convert_data_to_tensors(X=x, y=last_pos)
    
    last_pos2 = pd.Series(
        net(x_tensor).reshape(-1).detach().numpy(), 
        index=last_pos.columns
    )

    return last_pos2

last_position = pd.DataFrame(0.0, index=[calc_date], columns=ASSETS)
alloc = get_allocation(net=my_net, X=X_raw, last_pos=last_position)
alloc

VUSA.L    0.870272
CNX1.L    0.125078
CASH      0.004650
dtype: float32